In [1]:
import pandas as pd
import numpy as ny
import seaborn as sns

In [2]:
%pwd

'/Users/lyndon/Documents/Kaggle Project/predict-future-sales/Kaggle_predict_future_sales'

In [5]:
base= '/Users/lyndon/Documents/Kaggle Project/predict-future-sales/'

In [6]:
sales_train = pd.read_csv(base+'sales_train.csv')
item_cat = pd.read_csv(base+'item_categories.csv')
item = pd.read_csv(base+'items.csv')
shops = pd.read_csv(base+'shops.csv')

In [8]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [10]:
# column explaination
# ID - an Id that represents a (Shop, Item) tuple within the test set
# shop_id - unique identifier of a shop
# item_id - unique identifier of a product
# item_category_id - unique identifier of item category
# item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
# item_price - current price of an item
# date - date in format dd/mm/yyyy
# date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
# item_name - name of item
# shop_name - name of shop
# item_category_name - name of item category

In [9]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [13]:
print('There are %d items in total' %sales_train['item_id'].nunique())
print('There are %d shops in total' %sales_train['shop_id'].nunique())

There are 21807 items in total
There are 60 shops in total


In [20]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [16]:
item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [18]:
sample_sub = pd.read_csv(base+'sample_submission.csv')
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [19]:
test = pd.read_csv(base+'test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [28]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [26]:
# create a dict: 
# key:(shop_id, item_id)
# value: ID
keys = zip(test['shop_id'],test['item_id'])
values = test['ID']
ID_dict = dict(zip(keys,values))

In [47]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID
0,02.01.2013,0,59,22154,999.00,1.0,NaN
1,03.01.2013,0,25,2552,899.00,1.0,NaN
2,05.01.2013,0,25,2552,899.00,-1.0,NaN
3,06.01.2013,0,25,2554,1709.05,1.0,NaN
4,15.01.2013,0,25,2555,1099.00,1.0,NaN


In [50]:
# put all related columns into one dataset
sales_train['ID'] = [ID_dict.get(x,None) for x in list(zip(sales_train['shop_id'],sales_train['item_id']))]

In [51]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID
0,02.01.2013,0,59,22154,999.00,1.0,150983.0
1,03.01.2013,0,25,2552,899.00,1.0,NaN
2,05.01.2013,0,25,2552,899.00,-1.0,NaN
3,06.01.2013,0,25,2554,1709.05,1.0,NaN
4,15.01.2013,0,25,2555,1099.00,1.0,NaN


In [54]:
sales_w_ID = sales_train[~sales_train['ID'].isnull()]

In [55]:
len(sales_train)

2935849

In [56]:
len(sales_w_ID)

1224439

In [ ]:
# Question1: Almost half of the training set is missing ID, which means we do not need to predict those items so we might directly throw out these points for convenience
# However, we might not throw out these items because probably there are some correlation between items so they will provide more information.